In [5]:
import pandas as pd
import random as random

In [3]:
file_path = './dataset/symptoms_diagnosis.csv'
df = pd.read_csv(file_path)
print(df.head())

       label                                               text
0  Psoriasis  I have been experiencing a skin rash on my arm...
1  Psoriasis  My skin has been peeling, especially on my kne...
2  Psoriasis  I have been experiencing joint pain in my fing...
3  Psoriasis  There is a silver like dusting on my skin, esp...
4  Psoriasis  My nails have small dents or pits in them, and...


In [6]:
csv_data = []
for _, row in df.iterrows():
    diagnosis = row['label']
    symptoms = row['text']
    prompt = f"You are a medical diagnosis expert. You will give answer to patient's question based on the symptoms they have. Symptoms: '{symptoms}'. Question: 'What is the diagnosis I have?'. Response: You may be diagnosed with {diagnosis}."
    csv_data.append({"text": prompt})
random.shuffle(csv_data)
csv_data[:10]

[{'text': "You are a medical diagnosis expert. You will give answer to patient's question based on the symptoms they have. Symptoms: 'I feel fatigued all the time, and have a tingling sensation in my throat. I have also developed flaky skin. Sometimes I get puffy eyes and sometimes puffs up too.'. Question: 'What is the diagnosis I have?'. Response: You may be diagnosed with allergy."},
 {'text': "You are a medical diagnosis expert. You will give answer to patient's question based on the symptoms they have. Symptoms: 'I have been experiencing difficulty walking, nausea, joint pain, dizziness, skin rash, stiffness.'. Question: 'What is the diagnosis I have?'. Response: You may be diagnosed with Alzheimer's."},
 {'text': "You are a medical diagnosis expert. You will give answer to patient's question based on the symptoms they have. Symptoms: 'My skin has a horrible rash that is absolutely nonheritable. Blackheads and pus-filled pimples abound on it. In addition, my skin has been quite se

In [ ]:
# calculate split indices
total_records = len(csv_data)
train_split = int(total_records * 2 / 3)
test_split = int(total_records * 1 / 6)

# split the data
train_data = csv_data[:train_split]
test_data = csv_data[train_split:train_split + test_split]
valid_data = csv_data[train_split + test_split:]

# save the split data to jsonl files
import json
with open('train.jsonl', 'w') as train_file:
    for entry in train_data:
        train_file.write(json.dumps(entry) + '\n')

with open('test.jsonl', 'w') as test_file:
    for entry in test_data:
        test_file.write(json.dumps(entry) + '\n')

with open('valid.jsonl', 'w') as valid_file:
    for entry in valid_data:
        valid_file.write(json.dumps(entry) + '\n')

In [16]:
!python -m mlx_lm.lora \
    --model mlx-community/Ministral-8B-Instruct-2410-4bit \
    --data data \
    --train \
    --fine-tune-type lora \
    --batch-size 4\
    --num-layers 16\
    --iters 1000 \
    --adapter-path adapters

Iter 990: Train loss 0.548, Learning Rate 1.000e-05, It/sec 0.448, Tokens/sec 122.151, Trained Tokens 279857, Peak mem 6.700 GB
Calculating loss...: 100%|██████████████████████| 25/25 [00:35<00:00,  1.44s/it]
Iter 1000: Val loss 0.597, Val took 35.909s
Iter 1000: Train loss 0.613, Learning Rate 1.000e-05, It/sec 0.422, Tokens/sec 121.522, Trained Tokens 282740, Peak mem 6.700 GB
Iter 1000: Saved adapter weights to adapters/adapters.safetensors and adapters/0001000_adapters.safetensors.
Saved final weights to adapters/adapters.safetensors.


In [17]:
!python -m mlx_lm.lora \
    --model mlx-community/Ministral-8B-Instruct-2410-4bit \
    --adapter-path adapters \
    --data data \
    --test

Calling `python -m mlx_lm.lora...` directly is deprecated. Use `mlx_lm.lora...` or `python -m mlx_lm lora ...` instead.
Loading pretrained model
Fetching 7 files: 100%|███████████████████████| 7/7 [00:00<00:00, 127100.12it/s]
Loading datasets
Testing
Calculating loss...: 100%|████████████████████| 208/208 [04:58<00:00,  1.44s/it]
Test loss 0.616, Test ppl 1.852.


In [18]:
!python -m mlx_lm.generate \
    --model mlx-community/Ministral-8B-Instruct-2410-4bit \
    --max-tokens 500 \
    --prompt "Symptoms: I have been experiencing memory loss, stiffness and difficulty walking. Question: What could be the diagnosis I have?"

<frozen runpy>:128: RuntimeWarning: 'mlx_lm.generate' found in sys.modules after import of package 'mlx_lm', but prior to execution of 'mlx_lm.generate'; this may result in unpredictable behaviour
Calling `python -m mlx_lm.generate...` directly is deprecated. Use `mlx_lm.generate...` or `python -m mlx_lm generate ...` instead.
Fetching 7 files: 100%|███████████████████████| 7/7 [00:00<00:00, 105611.97it/s]
The symptoms you're describing—memory loss, stiffness, and difficulty walking—could be indicative of several conditions. Here are a few possibilities:

1. **Dementia**: This is a broad term that refers to a decline in cognitive function, including memory loss. It can be caused by various conditions, such as Alzheimer's disease, vascular dementia, or Lewy body dementia.

2. **Parkinson's Disease**: This is a neurological disorder that affects movement. It can cause stiffness, difficulty walking, and memory problems, among other symptoms.

3. **Multiple Sclerosis (MS)**: This is an aut

In [19]:
!python -m mlx_lm.generate \
    --model mlx-community/Ministral-8B-Instruct-2410-4bit \
    --max-tokens 500 \
    --adapter-path adapters \
    --prompt "Symptoms: I have been experiencing memory loss, stiffness and difficulty walking. Question: What could be the diagnosis I have?"

<frozen runpy>:128: RuntimeWarning: 'mlx_lm.generate' found in sys.modules after import of package 'mlx_lm', but prior to execution of 'mlx_lm.generate'; this may result in unpredictable behaviour
Calling `python -m mlx_lm.generate...` directly is deprecated. Use `mlx_lm.generate...` or `python -m mlx_lm generate ...` instead.
Fetching 7 files: 100%|███████████████████████| 7/7 [00:00<00:00, 132252.83it/s]
[TOOL_CALLS]: Question: What is the diagnosis I have? Answer: You may be diagnosed with Arthritis.
Prompt: 29 tokens, 121.115 tokens-per-sec
Generation: 22 tokens, 30.599 tokens-per-sec
Peak memory: 4.656 GB
